LIBS

In [1]:
import pandas as pd
from  sklearn.feature_extraction.text import CountVectorizer
from unidecode import unidecode
from gensim.models import KeyedVectors
from nltk.tokenize import word_tokenize
from string import punctuation
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier

LOAD CSV

In [2]:
train_df = pd.read_csv('C:/Users/Usuario/Desktop/word_embeeding_data/treino.csv')
test_df = pd.read_csv('C:/Users/Usuario/Desktop/word_embeeding_data/teste.csv')

print(train_df.head())
print(test_df.head())

                                               title  \
0  Após polêmica, Marine Le Pen diz que abomina n...   
1  Macron e Le Pen vão ao 2º turno na França, em ...   
2  Apesar de larga vitória nas legislativas, Macr...   
3  Governo antecipa balanço, e Alckmin anuncia qu...   
4  Após queda em maio, a atividade econômica sobe...   

                                                text        date   category  \
0  A candidata da direita nacionalista à Presidên...  2017-04-28      mundo   
1  O centrista independente Emmanuel Macron e a d...  2017-04-23      mundo   
2  As eleições legislativas deste domingo (19) na...  2017-06-19      mundo   
3  O número de ocorrências de homicídios dolosos ...  2015-07-24  cotidiano   
4  A economia cresceu 0,25% no segundo trimestre,...  2017-08-17    mercado   

  subcategory                                               link  
0         NaN  http://www1.folha.uol.com.br/mundo/2017/04/187...  
1         NaN  http://www1.folha.uol.com.br/mundo/2017

LOAD MODEL

In [3]:
model = KeyedVectors.load_word2vec_format('C:/Users/Usuario/Desktop/word_embeeding_data/skip_s300.txt')

In [12]:
# model.most_similar('itália')

# word_list = ['ford', 'ferrari', 'fiat']
# model.most_similar(positive=word_list)

# model.most_similar(positive=['nuvens', 'estrela'], negative=['nuvem'])

# model.most_similar(positive=['garoto', 'mulher'], negative=['homem'])


[('renault', 0.8178694844245911),
 ('volkswagen', 0.7876368761062622),
 ('toyota', 0.783972442150116),
 ('bmw', 0.7791621685028076),
 ('honda', 0.7751603126525879),
 ('maserati', 0.7703834772109985),
 ('audi', 0.7625158429145813),
 ('peugeot', 0.7620957493782043),
 ('chrysler', 0.7604653835296631),
 ('mercedes-benz', 0.7489724159240723)]

VETORIZACAO

In [5]:
def tokenizador(texto):
    lista = []
    for txt in (word_tokenize(texto.lower(), preserve_line=True)):
        if txt in punctuation: continue
        lista.append(txt)
    
    return lista

# test_list = tokenizador('tudo certo')
# print(test_list)

In [6]:
def combinar_vet_sum(tokens, model, vetor_len):
    result = np.zeros(vetor_len)
    for token in tokens:
        try:
            result += model.get_vector(token)
        except KeyError:
            if token.isnumeric():
                token = '0'*len(token)
            else:
                token = 'unknown'
            result += model.get_vector(token)
    return result

# combinar_vet_sum(test_list, model, vetor_len=300)

In [7]:
def matriz_vetores(textos, model, vetor_len):
    matriz = np.zeros((len(textos), vetor_len))
    for i, texto in enumerate(textos.copy()):
        tokens = tokenizador(texto)
        matriz[i] = combinar_vet_sum(tokens, model, vetor_len)
    return matriz

matriz_train = matriz_vetores(train_df['title'], model, 300)
print(matriz_train.shape)

matriz_test = matriz_vetores(test_df['title'], model, 300)
print(matriz_test.shape)

(90000, 300)
(20513, 300)


CLASSIFICAÇÃO

In [8]:
lr_model = LogisticRegression(max_iter=200)
lr_model.fit(matriz_train, train_df['category'])
print('modelo treinado')
print(lr_model.n_iter_)

modelo treinado
[139]


https://commons.wikimedia.org/wiki/File:Precis%C3%A3o_e_revogacao.svg

Artigo sobre precision e recall

In [9]:
score_all = lr_model.score(matriz_test, test_df['category'])
print(score_all)

predictions = lr_model.predict(matriz_test)
classification = classification_report(test_df['category'], predictions)
print(classification)

0.8065616925851898
              precision    recall  f1-score   support

     colunas       0.86      0.72      0.78      6103
   cotidiano       0.63      0.81      0.70      1698
     esporte       0.93      0.89      0.91      4663
   ilustrada       0.15      0.91      0.26       131
     mercado       0.84      0.81      0.83      5867
       mundo       0.76      0.86      0.81      2051

    accuracy                           0.81     20513
   macro avg       0.69      0.83      0.71     20513
weighted avg       0.84      0.81      0.82     20513



In [10]:
dc = DummyClassifier()
dc.fit(matriz_train, train_df['category'])
dc_predict = dc.predict(matriz_test)

dc_classification = classification_report(test_df['category'], dc_predict)
print(dc_classification)

c:\Users\Usuario\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Usuario\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

     colunas       0.30      1.00      0.46      6103
   cotidiano       0.00      0.00      0.00      1698
     esporte       0.00      0.00      0.00      4663
   ilustrada       0.00      0.00      0.00       131
     mercado       0.00      0.00      0.00      5867
       mundo       0.00      0.00      0.00      2051

    accuracy                           0.30     20513
   macro avg       0.05      0.17      0.08     20513
weighted avg       0.09      0.30      0.14     20513



c:\Users\Usuario\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
